# EXAM 1

This activity is intended to assess your knowledge
in formulating engineering problems as univariate root-finding tasks,
as well as in applying relevant numerical methods to solve them.

## SCENARIO

Two batching plants -- Bonnie and Clyde --
have respective ratings of
$p_{1}$ kilowatts and $q_{2}$ kilovars,
have respective lagging power factors of
$\phi_{1}$ and $\phi_{2}$,
and
are separately served by dedicated feeders
whose respective impedances are
$r_{1} + j x_{1}$
and
$r_{2} + j x_{2}$ ohms.
The feeders branch out from a substation bus
regulated around a nominal voltage of $u$ kilovolts.

The main objective is to determine the phase-angle difference
between the rated operating voltages of Bonnie and of Clyde.
One way to accomplish the main objective is
to solve for the operating voltage magnitudes of Bonnie and of Clyde
(say, $v_{1}$ and $v_{2}$, respectively).
With such values,
determining the desired phase-angle difference
is a matter of analysis and complex-number arithmetic.

## PART I

This is the Written component of the exam.

#### QUESTION 1
  Derive a univariate function $f\!\left(v\right)$ whose root is $v_{1}$.
  
#### QUESTION 2
  Derive a univariate function $g\!\left(v\right)$ whose root is $v_{2}$.

#### QUESTION 3
  Describe how to calculate the desired phase-angle difference using $v_{1}$ and $v_{2}$.

#### QUESTION 4
  If you are to solve for $v_{1}$ using the bisection method, how will you set the search interval?

#### QUESTION 5
  If you are to solve for $v_{2}$ using the Newton-Raphson method, how will you set the initial estimate?

#### QUESTION 6
  If you are to solve for $v_{1}$ using the Newton-Raphson method, derive an iteration scheme of the form $v_{k+1} \gets v_{k} + h\!\left(v_{k}\right)$.

#### QUESTION 7
  Reason out whether or not the desired phase-angle difference can take on multiple values.

#### QUESTION 8
  In your stay so far in the university, what do you consider to be the most profound thing you have heard or learned from a teacher?


## PART II

This is the Programming component of the exam.

Set the power ratings as follows.
- M34W12 class:
  $p_{1} = 450$, $\phi_{1} = 0.87$, $q_{2} = 300$, and $\phi_{2} = 0.90$
- M34W34 class:
  $p_{1} = 375$, $\phi_{1} = 0.95$, $q_{2} = 325$, and $\phi_{2} = 0.80$
- H34W56 class:
  $p_{1} = 425$, $\phi_{1} = 0.75$, $q_{2} = 275$, and $\phi_{2} = 0.75$
- H34W78 class:
  $p_{1} = 301$, $\phi_{1} = 0.73$, $q_{2} = 334$, and $\phi_{2} = 0.84$

Set the feeder line parameters as follows.
- M34W12 class:
  $r_{1} = 0.97$, $x_{1} = 2.45$, $r_{2} = 0.99$, and $x_{2} = 2.51$
- M34W34 class:
  $r_{1} = 0.95$, $x_{1} = 2.54$, $r_{2} = 1.01$, and $x_{2} = 2.75$
- H34W56 class:
  $r_{1} = 1.01$, $x_{1} = 2.75$, $r_{2} = 0.95$, and $x_{2} = 2.54$
- H34W78 class:
  $r_{1} = 0.99$, $x_{1} = 2.51$, $r_{2} = 0.97$, and $x_{2} = 2.45$

Adopting real-world conditions,
the substation bus voltage is allowed to vary within 10 % of the nominal.
(Exceeding this range indicates undervoltage or overvoltage.)
Set the substation bus voltage as follows.
- M34W12 class: the minimum allowed for a $u = 49.0$
- M34W34 class: the maximum allowed for a $u = 13.8$
- H34W56 class: the minimum allowed for a $u = 13.8$
- H34W78 class: the maximum allowed for a $u = 49.0$

#### Using the values for H34W56 class

Set the power ratings as follows.
- H34W56 class:
  $p_{1} = 425$, $\phi_{1} = 0.75$, $q_{2} = 275$, and $\phi_{2} = 0.75$
  
Set the feeder line parameters as follows.
- H34W56 class:
  $r_{1} = 1.01$, $x_{1} = 2.75$, $r_{2} = 0.95$, and $x_{2} = 2.54$
  
Adopting real-world conditions, the substation bus voltage is allowed to vary within 10 % of the nominal.
(Exceeding this range indicates undervoltage or overvoltage.)

Set the substation bus voltage as follows.
- H34W56 class: the minimum allowed for a $u = 13.8$

### PRELIMINARIES

In [1]:
import math as mt
import scipy.optimize as spo

Define the following Python functions.
- `f()`, implementing $f\!\left(v\right)$
- `g()`, implementing $g\!\left(v\right)$
- `df()`, implementing the derivative of $f\!\left(v\right)$ w.r.t. $v$
- `dg()`, implementing the derivative of $g\!\left(v\right)$ w.r.t. $v$

Each of these functions should have the following inputs.
- `v`,
  a positional argument that represents the operating voltage
- `v_ss`,
  a keyword argument that represents the substation bus voltage
  and defaults to the nominal value

Lastly, define a Python function
`calc_angle_diff()` that returns the desired phase-angle difference
(in radians)
given two inputs:
- `v1`, the first positional argument representing $v_{1}$,
  and
- `v2`, the first positional argument representing $v_{2}$.

Replace the `pass` statements in the succeeding code cells.

In [2]:
p1 = 425      # kW
phi1 = 0.75   # pf
q2 = 275      # kvar
phi2 = 0.75   # pf

r1, x1 = 1.01, 2.75
r2, x2 = 0.95, 2.54

u = 13.8                # nominal kV
v_ss = u/0.9          # minimum allowed (since "minimum allowed for u = 13.8")

print(f"Substation voltage v_ss = {v_ss:.3f} kV")

# Compute reactive and active components per plant
q1 = p1 * mt.tan(mt.acos(phi1))
p2 = q2 / mt.tan(mt.acos(phi2))

Substation voltage v_ss = 15.333 kV


In [3]:
def v_ss(u):
    return u/0.9

In [4]:
def f(v):
    return (v+(1.459990615/v))**2 + (0.7901870832/v)**2 - (v_ss(u))**2

In [5]:
def g(v):
    return (v+(0.9947296557/v))**2 + (0.5307745532/v)**2 - (v_ss(u))**2

In [6]:
def df(v):
    return 2*v - (5.511936445/(v**3))

In [7]:
def dg(v):
    return 2*v - (2.542417429/(v**3))

In [8]:
def calc_angle_diff(v1, v2):
    return abs(mt.degrees(mt.atan(0.7901870832/1.459990615) - mt.atan(0.5307745532/0.9947296557)))

#### Using the bisection method

In solving for $v_{1}$,
define Python variables `_V1L` and `_V1U`
to store the lower and the upper ends of the search interval.
Similarly define Python variables `_V2L` and `_V2U`
in solving for $v_{2}$.

Run 
[`scipy.optimize.bisect()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.bisect.html)
such that you get
the (approximate) root
as well as information concerning the run.
Store the (approximate) root and the information in Python variables

- `v1_bs` and `v1_bs_info`, respectively, for $v_{1}$; and
- `v2_bs` and `v2_bs_info`, respectively, for $v_{2}$.

Set tolerances and iteration budget to their corresponding default values.

Replace the `pass` statement in the succeeding cells with the appropriate code.

In [9]:
_V1L, _V1U = 13.8, 16.87
_V2L, _V2U = 13.8, 16.87
X_TOL = 1e-7
MAX_ITERS = 750

v1_bs, v1_bs_info = spo.bisect(f, _V1L, _V1U, rtol = X_TOL, maxiter = MAX_ITERS, full_output = True, disp = False)

v2_bs, v2_bs_info = spo.bisect(g, _V2L, _V2U, rtol = X_TOL, maxiter = MAX_ITERS, full_output = True, disp = False)


In [10]:
print("-----")
print(v1_bs_info)
print("-----")
print(v2_bs_info)

-----
      converged: True
           flag: converged
 function_calls: 23
     iterations: 21
           root: 15.237430262565615
         method: bisect
-----
      converged: True
           flag: converged
 function_calls: 23
     iterations: 21
           root: 15.268142676353456
         method: bisect


In [11]:
print(f"Solving for Bonnie's operating voltage using the bisection method...")
print(f"\tinterval: {_V1L} - {_V1U} kilovolts")
print(f"\troot: {v1_bs}")
print(f"\tresidual: {f(v1_bs)}")

Solving for Bonnie's operating voltage using the bisection method...
	interval: 13.8 - 16.87 kilovolts
	root: 15.237430262565615
	residual: 2.11261146034758e-05


In [12]:
print(f"Solving for Clyde's operating voltage using the bisection method...")
print(f"\tinterval: {_V2L} - {_V2U} kilovolts")
print(f"\troot: {v2_bs}")
print(f"\tresidual: {g(v2_bs)}")

Solving for Clyde's operating voltage using the bisection method...
	interval: 13.8 - 16.87 kilovolts
	root: 15.268142676353456
	residual: -1.7901919079577056e-05


In [13]:
print(f"Phase-angle difference: {calc_angle_diff(v1_bs, v2_bs)} degrees")

Phase-angle difference: 0.33968148684423793 degrees


#### Using the Newton-Raphson method

In solving for $v_{1}$,
define a Python variable `_V1NR`
to store the initial estimate.
Similarly define a Python variable `_V2NR`
in solving for $v_{2}$.

Run
[`scipy.optimize.newton()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.newton.html)
such that you get the (approximate) root as well as convergence information.
Store the (approximate) root and the information in Python variables

- `v1_nr` and `v1_nr_info`, respectively, for $v_{1}$; and
- `v2_nr` and `v2_nr_info`, respectively, for $v_{2}$.

Set tolerances and iteration budget to their corresponding default values.

Replace the `pass` statement in the succeeding cell with your code.

In [14]:
_V1NR, _V2NR = u, u
X_TOL = 1e-7
MAX_ITERS = 100


v1_nr, v1_nr_info = spo.newton(f, _V1NR, fprime=df, args=(), tol=X_TOL, maxiter=MAX_ITERS, fprime2=None, x1=None, rtol=0.0, full_output=True, disp=False)

v2_nr, v2_nr_info = spo.newton(g, _V2NR, fprime=dg, args=(), tol=X_TOL, maxiter=MAX_ITERS, fprime2=None, x1=None, rtol=0.0, full_output=True, disp=False)


In [15]:
print("-----")
print(v1_nr_info)
print("-----")
print(v2_nr_info)

-----
      converged: True
           flag: converged
 function_calls: 8
     iterations: 4
           root: 15.23742956929927
         method: newton
-----
      converged: True
           flag: converged
 function_calls: 8
     iterations: 4
           root: 15.268143262617873
         method: newton


In [16]:
print(f"Solving for Bonnie's operating voltage using the Newton-Raphson method...")
print(f"\tstart: {_V1NR} kilovolts")
print(f"\troot: {v1_nr}")
print(f"\tresidual: {f(v1_nr)}")

Solving for Bonnie's operating voltage using the Newton-Raphson method...
	start: 13.8 kilovolts
	root: 15.23742956929927
	residual: 0.0


In [17]:
print(f"Solving for Clyde's operating voltage using the newton-raphson method...")
print(f"\tstart: {_V2NR} kilovolts")
print(f"\troot: {v2_nr}")
print(f"\tresidual: {g(v2_nr)}")

Solving for Clyde's operating voltage using the newton-raphson method...
	start: 13.8 kilovolts
	root: 15.268143262617873
	residual: -2.842170943040401e-14


In [18]:
print(f"Phase-angle difference: {calc_angle_diff(v1_nr, v2_nr)} degrees")

Phase-angle difference: 0.33968148684423793 degrees


#### Using the secant method

In solving for $v_{1}$,
define a Python variable `_V1SC`
to store the initial estimate.
Similarly define a Python variable `_V2SC`
in solving for $v_{2}$.

Run
[`scipy.optimize.newton()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.newton.html)
such that you get the (approximate) root as well as information concerning the run.
Store the (approximate) root and the information in Python variables

- `v1_sc` and `v1_sc_info`, respectively, for $v_{1}$; and
- `v2_sc` and `v2_sc_info`, respectively, for $v_{2}$.

Set tolerances and iteration budget to their corresponding default values.

Replace the `pass` statement in the succeeding cell with your code.

In [19]:
_V1SC, _V2SC = u, u
MAX_ITERS = 750
X_TOL = 1e-7


v1_sc, v1_sc_info = spo.newton(f, _V1SC, fprime=None, args=(), tol=X_TOL, maxiter=MAX_ITERS, fprime2=None, x1=None, rtol=0.0, full_output=True, disp=False)

v2_sc, v2_sc_info = spo.newton(g, _V2SC, fprime=None, args=(), tol=X_TOL, maxiter=MAX_ITERS, fprime2=None, x1=None, rtol=0.0, full_output=True, disp=False)


In [20]:
print("-----")
print(v1_sc_info)
print("-----")
print(v2_sc_info)

-----
      converged: True
           flag: converged
 function_calls: 6
     iterations: 5
           root: 15.237429569299268
         method: secant
-----
      converged: True
           flag: converged
 function_calls: 6
     iterations: 5
           root: 15.268143262617873
         method: secant


In [21]:
print(f"Solving for Bonnie's operating voltage using the secant method...")
print(f"\tstart: {_V1SC} kilovolts")
print(f"\troot: {v1_sc}")
print(f"\tresidual: {f(v1_sc)}")

Solving for Bonnie's operating voltage using the secant method...
	start: 13.8 kilovolts
	root: 15.237429569299268
	residual: -5.684341886080802e-14


In [22]:
print(f"Solving for Clyde's operating voltage using the secant method...")
print(f"\tstart: {_V2SC} kilovolts")
print(f"\troot: {v2_sc}")
print(f"\tresidual: {g(v2_sc)}")

Solving for Clyde's operating voltage using the secant method...
	start: 13.8 kilovolts
	root: 15.268143262617873
	residual: -2.842170943040401e-14


In [23]:
print(f"Phase-angle difference: {calc_angle_diff(v1_sc, v2_sc)} degrees")

Phase-angle difference: 0.33968148684423793 degrees


### INSTRUCTIONS AND REMINDERS

### Written exam component

Write your answers in white A4 papers.
Use one (1) inch for the top, left, right, and bottom margins.
Write only on one page of a sheet.
Staple the sheets at the upper-left corner of the pages.

Corresponding maximum marks for Questions 1 - 7 are as follows.
- thirty (30) points for each of Questions 1 and 2
- ten (10) points for Question 3
- five (5) points for each of Questions 4 - 7

Question 8 is a bonus worth three (3) points.

The use of AI tools to answer this exam is not prohibited,
but it is of ethical interest to disclose such use.
This is in line with the
[MSU Policy on the Fair and Ethical Use of AI and Its Applications](https://www.msumain.edu.ph/wp-content/uploads/2024/05/MSU-Policy-on-Ethical-use-of-AI-Policies.pdf).
As such, include a brief closing section
(titled "Declaration on the use of AI tools")
declaring which and how AI tools are used in your work,
or non-use thereof.

### Programming component

Do not use any library or module other than those in the imports cell.

In addition to replacing the `pass` statements,
you also need to fix some intentional errors.

For each method,
- getting the correct value of $v_{1}$ merits ten (10) points,
- getting the correct value of $v_{2}$ merits ten (10) points,
- getting the correct value of the phase-angle difference merits ten (10) points,
  and
- getting a reasonable residual merits two (2) points.

Meeting the above conditions and all instructions
further merits a point.
Each non-compliance to an instruction, however, means a deduction of two (2) points.
Thus, one may earn up to 100 points for the programming component,
as long as
- all instructions are met,
- all code cells run properly in succession,
  and
- all computed quantities are (acceptably) equal to those in a held-out answer key.

Download this notebook file,
and save with a filename following the pattern
`EXM-01_<section>_<ID number>`,
where the section is as reflected in your Google Classroom.
For example,
if your ID number is 2013-0024
and you are enrolled in the M34W12 class,
then your notebook should be named
`EXM-01_M34W12_2013-0024.ipynb`.
Submit your notebook via the classwork platform for Exam 1 in Google Classroom.
Submissions beyond the deadline will not be considered.

The use of AI tools to answer this exam is not prohibited,
but it is of ethical interest to disclose such use.
This is in line with the
[MSU Policy on the Fair and Ethical Use of AI and Its Applications](https://www.msumain.edu.ph/wp-content/uploads/2024/05/MSU-Policy-on-Ethical-use-of-AI-Policies.pdf).
As such, please include a brief statement
(in a private comment to this classwork)
declaring which and how AI tools are used in your work,
or non-use thereof.

*Last updated by Christian Cahig on 2025-10-22*